In [1]:
from HANEmotionAnalyze import *
import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer
import numpy
import time, math
import torch.utils.data as data
import os
import pandas as pd
import json
import thulac
thulac = thulac.thulac()
import jieba

Model loaded succeed


In [2]:
# import torch.functional as F
# 
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([[0, 1,2,4,5],[1, 4,3,2,9]])
print(embedding(input))
embedding2 = nn.Embedding(10, 3, padding_idx=2)
# input = torch.LongTensor([[1,2,0,5, 6,7,8,9]])
print(embedding2(input))
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding3 = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([1])
print(embedding3(input))
input = torch.LongTensor([0])
print(embedding3(input))



tensor([[[-1.0199, -0.6915, -1.6168],
         [ 0.7839, -0.0831,  0.2477],
         [ 1.2109,  0.8160,  2.6015],
         [ 0.6143,  1.8320, -0.2086],
         [-1.4623,  1.1073, -0.5561]],

        [[ 0.7839, -0.0831,  0.2477],
         [ 0.6143,  1.8320, -0.2086],
         [-0.2993,  0.0033,  1.5736],
         [ 1.2109,  0.8160,  2.6015],
         [-0.0288,  1.0980, -1.2566]]], grad_fn=<EmbeddingBackward>)
tensor([[[-0.2129,  0.4830,  0.2750],
         [-0.7534, -0.2165,  0.4123],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.6356, -0.4989, -0.3891],
         [ 0.0896, -1.6047, -0.6569]],

        [[-0.7534, -0.2165,  0.4123],
         [ 0.6356, -0.4989, -0.3891],
         [-1.1882, -0.4552, -0.3379],
         [ 0.0000,  0.0000,  0.0000],
         [-1.1927,  1.1059,  0.1896]]], grad_fn=<EmbeddingBackward>)
tensor([[4.0000, 5.1000, 6.3000]])
tensor([[1.0000, 2.3000, 3.0000]])


## 读取词向量
建立词语列表

In [3]:
from gensim.models import KeyedVectors, Word2Vec

# file = '../../PretrainedData/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'
# file = '../../DataSets/Word2Vect/xingrong_50_thulac/word2vect_50_w5.model'
file = '../../DataSets/Word2Vect/xiejunjie_300_jieba/wiki_han_word2vec_300维度.model'
word2vec = Word2Vec.load(file)
# word2vec = KeyedVectors.load_word2vec_format(file, binary=False)
word2vec.init_sims(replace=True)  # 神奇，很省内存，可以运算most_similar
word2vec.vector_size

unable to import 'smart_open.gcs', disabling that module


300

In [4]:
print(word2vec)
# print(word2vec.wv.vocab)
# print(len(word2vec.index2word))
print(len(word2vec.wv.index2word))
print(word2vec.wv.index2word[0])
print(word2vec.wv.index2word[1])
print(word2vec.wv.index2word[2])
print(word2vec.wv.index2word[1522])
print(word2vec.wv.index2entity[1522])
print(word2vec.similar_by_word('中国'))
print(word2vec.similar_by_word('天才'))
print(word2vec.wv)
print('word2vec.wv.vocab ---- >', word2vec.wv.vocab)
print(word2vec.wv.index2word)


Word2Vec(vocab=109683, size=300, alpha=0.025)
109683

的
在
厅
厅
[('我国', 0.4018472731113434), ('中方', 0.39661142230033875), ('各国', 0.3935665488243103), ('欧美', 0.39242100715637207), ('强国', 0.3893248438835144), ('亚洲', 0.3873169422149658), ('国内', 0.38274306058883667), ('非洲', 0.36376285552978516), ('哈萨克斯坦', 0.36366286873817444), ('外国', 0.36008766293525696)]
[('死神', 0.6855337619781494), ('泰勒', 0.6751863956451416), ('天生', 0.6720190644264221), ('青梅竹马', 0.6718809008598328), ('英俊', 0.6717637181282043), ('杰克', 0.6692911386489868), ('女人', 0.6632643938064575), ('私生子', 0.6598134636878967), ('王子', 0.6573523283004761), ('安娜', 0.6552286744117737)]
word2vec.wv.vocab ---- > 

C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  # This is added back by InteractiveShellApp.init_path()
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
wordEmbedding = [word2vec.wv[word]  for word in word2vec.wv.index2word]
word2index = { word:i for i, word in enumerate(word2vec.wv.index2word)}
print(wordEmbedding[:10])
print(word2index['中国'])
print(word2index['天才'])


[array([ 6.30656257e-02, -1.12314887e-01,  4.85262945e-02, -8.55197757e-02,
        4.55908328e-02,  6.46558106e-02, -4.27534170e-02,  5.36949895e-02,
       -2.26631403e-01,  6.25260845e-02,  1.83971524e-02, -3.61814126e-02,
       -7.80684203e-02, -9.78453308e-02,  3.07323225e-02,  4.43015546e-02,
       -1.03984829e-02, -3.16940509e-02, -4.82366160e-02, -2.55376995e-02,
       -1.94135922e-04, -2.42569018e-02, -5.79886399e-02,  1.26771117e-02,
        7.19933212e-03, -5.69830649e-02, -1.26355989e-02,  2.16077268e-02,
       -8.15668255e-02,  9.75341350e-03,  4.03154232e-02, -2.12772768e-02,
        4.27430905e-02, -2.53562219e-02,  9.61699188e-02,  4.80885766e-02,
       -5.42805195e-02, -3.23644653e-02, -1.98951829e-02,  3.14243254e-03,
       -4.23267344e-03,  8.15014020e-02,  5.07650971e-02, -6.43669665e-02,
        5.04725389e-02,  2.95574106e-02, -1.39722880e-02, -9.62358862e-02,
       -4.40400690e-02, -2.26076338e-02, -4.84022610e-02,  6.48457836e-03,
       -5.65880351e-02, 

## 读取训练数据

In [6]:
def compute_ngrams(word, num_min = 1, num_max = 3):
    ngrams =[]
    for ngram_length in range(num_min, min(len(word), num_max) + 1):
        for i in range(len(word) - ngram_length + 1):
            # print(i, i + ngram_length)
            ngrams.append(word[i : i + ngram_length])
    # print(ngrams)
    return list(set(ngrams))

print(compute_ngrams('you'))
print(compute_ngrams('I Think'))
print(compute_ngrams('中华人民共和国万岁'))

['ou', 'y', 'yo', 'you', 'u', 'o']
['k', ' ', 'Th', 'I ', ' T', 'T', 'Thi', 'hin', 'ink', ' Th', 'h', 'n', 'I T', 'in', 'hi', 'I', 'i', 'nk']
['人', '和国万', '和国', '中', '华人民', '共', '民', '民共', '民共和', '华人', '人民', '中华人', '华', '国', '人民共', '国万', '国万岁', '共和国', '万', '共和', '中华', '万岁', '岁', '和']


In [7]:
# 从词向量文本文件 word2vec 中获取词向量，如果获取到直接返回，若没有获取到，那么把这个词拆开
# 成为 ngrams 的新词组，并在 word2vec 中找新词组中的词向量并相加取平均，最后得到平均词向量输出
def wordVec(word, word2vec, min_n = 1, max_n = 3):
    # 确认词向量维度
    word_size = word2vec.wv.syn0[0].shape[0]

    # 如果在词典之中，直接返回词向量
    if word in word2vec.wv.vocab.keys():
        return word2vec[word]
    else:
        # 计算word的ngrams词组
        ngrams = compute_ngrams(word, min_n, max_n)
        # 不在词典的情况下
        word_vec = numpy.zeros(word_size, dtype=numpy.float32)
        ngrams_found = 0
        ngrams_single = [ng for ng in ngrams if len(ng) == 1]
        ngrams_more = [ng for ng in ngrams if len(ng) > 1]
        # 先只接受2个单词长度以上的词向量
        for ngram in ngrams_more:
            if ngram in word2vec.wv.vocab.keys():
                word_vec += word2vec[ngram]
                ngrams_found += 1
                #print(ngram)
        # 如果，没有匹配到，那么最后是考虑单个词向量
        if ngrams_found == 0:
            for ngram in ngrams_single:
                word_vec += word2vec[ngram]
                ngrams_found += 1
        if word_vec.any():
            return word_vec / max(1, ngrams_found)
        else:
            # 不抛出异常，而是打印提示，并返回0向量。
            print(KeyError('all ngrams for word %s absent from model' % word))
            return word_vec

In [8]:
class DotDict(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self

In [14]:
def isNan(a):
    return a != a

class RatingData(data.Dataset):
    def __init__(self, path, 
                 word2index, 
                 max_row = -1, 
                 trainTestRate = 0.85, 
                 isTrain = True, 
                 wordCuter = thulac,
                 clean_file_name = 'ratings_clean_4_HAN.csv',
                 ):
        self.token_list = []
        self.label_list = []
        # self.token_positions = torch.tensor([i for i in range(100)])

        print(' ratings.csv 所在path:',path) # 地址不应该包含 ratings.csv

        ratings_clean_filename = os.path.join(path, clean_file_name)
        ratings_filename = os.path.join(path, 'ratings.csv')
        if os.path.isfile(ratings_clean_filename):
            clean_pd = pd.read_csv(ratings_clean_filename)
        else:
            print('没有找到缓存的文件%s, 读取源文件%s'%(ratings_clean_filename, ratings_filename))
            ratings_pd = pd.read_csv(ratings_filename)
            print('开始生成缓存文件%s'%(ratings_clean_filename))
            clean_pd = pd.DataFrame({
                'userId':[],
                'restId':[],
                'rating':[],
                'comment':[],
            })
            nonRatingCount = 0
            for i, row in ratings_pd.iterrows():
                if max_row != -1 and i > max_row:
                    break
                if not isinstance(row['comment'], str) or row['comment'] == '':
                    # print(i + 1, row['comment'])
                    nonRatingCount += 1
                    continue
                r0 = row['rating']
                r1 = row['rating_env']
                r2 = row['rating_flavor']
                r3 = row['rating_service']
                if r0 == '' or isNan(r0): r0 = 0 # 假设总评分为 0 表示未评分
                if r1 == '' or isNan(r1): r1 = 3
                if r2 == '' or isNan(r2): r2 = 3
                if r3 == '' or isNan(r3): r3 = 3
                r0 = round(r0 * 0.5 + (r1 + r2 + r3) * 0.1666666)
                if i % 10000 == 9999:
                    print(i + 1, r0)

                # token = tokenizer.encode(text=str(row['comment']), max_length=100, pad_to_max_length = True)
                # words = thulac.cut(row['comment'], text=True)
                words = list(wordCuter.cut(row['comment']))
                # 0 在词向量集中是‘，’，换个词向量集可能表示其他
                token = [ word2index[words[i]] if i < len(words) and words[i] in word2index else 0 
                          for i in range(100)] 
                
                newRow = DotDict()
                newRow.userId = [row['userId']]
                newRow.restId = [row['restId']]
                newRow.rating = [r0]
                newRow.comment = [json.dumps(token)]

                clean_pd = clean_pd.append(pd.DataFrame(newRow), ignore_index=True)
            print('空的评论数量： %d'%(nonRatingCount))
            clean_pd.to_csv(ratings_clean_filename)

        # 读取
        if isTrain:
            temp_pd = clean_pd[ : int(len(clean_pd) * trainTestRate)]
        else:
            temp_pd = clean_pd[int(len(clean_pd) * trainTestRate) : ]

        for i, row in temp_pd.iterrows():
            if max_row != -1 and i > max_row:
                break

            self.label_list.append(torch.tensor(row['rating']).long())
            self.token_list.append(torch.from_numpy(numpy.array( json.loads(row['comment']) ) ).long())

    def __getitem__(self, index):
        return self.token_list[index], self.label_list[index]#, self.token_positions

    def __len__(self):
        return len(self.label_list)
    

In [ ]:
ratingData = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             max_row= 200000,
                             isTrain=True,
                             wordCuter= jieba,
                             clean_file_name='ratings_clean_4_HAN_jieba.csv',
                             )
trainLoader = torch.utils.data.DataLoader(dataset=ratingData,
                                          batch_size=256,
                                          shuffle = True,
                                          # num_workers = 0,
                                          )
ratingData2 = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             max_row= 200000,
                             isTrain=False,
                             wordCuter= jieba,
                             clean_file_name='ratings_clean_4_HAN_jieba.csv',
                             )
testLoader = torch.utils.data.DataLoader(dataset=ratingData2,
                                          batch_size=256,
                                          shuffle = True,
                                          # num_workers = 0,
                                          )
print(len(ratingData.label_list))
print(len(ratingData2.label_list))


 ratings.csv 所在path: ../../DataSets/yf_dianping
没有找到缓存的文件../../DataSets/yf_dianping\ratings_clean_4_HAN_jieba.csv, 读取源文件../../DataSets/yf_dianping\ratings.csv


In [ ]:
#创建模型
wordEmbedding = torch.FloatTensor(wordEmbedding)
num_embeddings = len(word2vec.wv.index2word)
model = HAN(num_embeddings, 
            embedding_dim=word2vec.wv.vector_size, 
            num_words = 100,
            hidden_size_gru = 128,
            hidden_size_att = 512,
            )
print(model)

modelParams = model.parameters()
for param in modelParams:
    if len(param.data.shape) > 1:
        print('---', param.data.shape, param.data)
        torch.nn.init.kaiming_normal(param.data)
        print('--->', param.data)
        
model.embed.from_pretrained(wordEmbedding)


## 开始训练过程

In [ ]:
def trainOneEpoch(epoch, model:HAN, trainLoader, optimizer:Optimizer, lossFunc):
    if torch.cuda.is_available():
        model = model.cuda()
        lossFunc = lossFunc.cuda()

    model.train()

    startTime = time.time()
    for i, (x, y) in enumerate(trainLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        loss = lossFunc(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 99:
            print('Epoch %d, %d/%d, loss:%f ' % (epoch, i, len(trainLoader), loss))
        # if i > 2000:
        #     break 
    print('Epoch %d cost time: %.3fs' % (epoch, time.time() - startTime))


def testModel(epoch, model:HAN, testLoader):
    if torch.cuda.is_available():
        model = model.cuda()

    model.eval()

    total = 0
    correct = 0

    startTime = time.time()
    for i, (x, y) in enumerate(testLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)

        total += len(y)
        correct += predicted.data.eq(y.data).cpu().sum().numpy()
        

        if i % 50 == 49:
            print('Epoch Test %d, %d/%d, \npredicted:%s, \ntarget:%s' % (epoch, i, len(testLoader), 
                                                                     str(predicted),
                                                                     str(y)))
        # if i > 2000:
        #     break 
    print('Epoch Test %d cost time: %.3fs' % (epoch, time.time() - startTime))
    print('准确率： %.3f' % (correct / total))


def train(nepoch, model, modelSavePath):
    # optimizer=torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.001)
    optimizer=torch.optim.Adam(model.parameters(), lr=0.0001)
    lossFunc =torch.nn.CrossEntropyLoss()
    for epoch in range(nepoch):
        trainOneEpoch(epoch, model, trainLoader, optimizer, lossFunc)
        testModel(epoch, model, testLoader)
    torch.save(model.state_dict(), modelSavePath)

def eval(modelSavePath, isLoad = True):
    if isLoad: model.load_state_dict(torch.load(modelSavePath))
    testModel(0, model, testLoader)

train(10, model, 'EmotionAnalyzeModelData.model')

In [ ]:
lossFunc =torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
_, predicted = torch.max(input.data, 1)
output = lossFunc(input, target)
print('input',input, '\n target',target, '\n output', output)
output = lossFunc(input, predicted)
print('predicted',predicted, '\n target',target, '\n output', output)

print(predicted.data.eq(target).cpu().sum())
print(target.data.eq(predicted).cpu().sum())

In [ ]:
# 何凯明初始化

w = torch.Tensor(3, 5, 2)
print(w)
print(nn.init.kaiming_uniform(w))
print(w)
w = torch.Tensor(3, 5, 2)
print(w)
print(torch.nn.init.kaiming_normal(w))
print(w)